# SAMOS Spectroscopy - Interactive Trace Identification

## Status: 📝 Placeholder Notebook

This notebook is a **future enhancement** for interactive trace identification and fine-tuning.

### Current Workflow

For now, **automated trace detection is handled in Notebook 01** (`01_initial_inspection.ipynb`).

Notebook 01 uses the `samos.spectroscopy.trace_detection` module to:
- Automatically detect slit edges from arc lamp
- Resolve overlapping slits
- Define extraction regions with sky margins
- Extract individual slit cutouts

### Why Automated Works Well

The automated trace detection in Notebook 01:
- ✅ Finds all traces reliably
- ✅ Handles closely-spaced slits
- ✅ Provides consistent results
- ✅ Requires minimal user input

### Future Enhancements for This Notebook

When implemented, this notebook will provide:

1. **Interactive Trace Adjustment**
   - Manually add/remove traces
   - Adjust slit boundaries
   - Fix problematic edge detections
   - Visual feedback with overlays

2. **Trace Quality Assessment**
   - Signal-to-noise per trace
   - Trace curvature visualization
   - Flag bad/unusable traces

3. **Alternative Detection Methods**
   - Gaussian fitting to spatial profile
   - Cross-correlation with template
   - Manual trace definition

4. **Parameter Optimization**
   - Interactive sliders for threshold adjustment
   - Real-time preview of detection results
   - Save optimized parameters to config file

### What to Do Now

**Use Notebook 01** - The automated trace detection works well for standard observations.

If you encounter issues with trace detection:

1. **Adjust parameters in Notebook 01**:
   ```python
   slits = trace_detection.detect_traces_from_arc(
       arc_good,
       x_range=(1500, 2000),      # Change this range
       peak_threshold=4e5,         # Lower to detect fainter slits
       margin=12,                  # Adjust sky margin
       verbose=True
   )
   ```

2. **Use the trace_detection module directly**:
   See example below

3. **Manually define slits**:
   If automatic detection fails completely

### Example: Advanced Trace Detection

In [ ]:
from samos.core import mosaic, cosmic_rays
from samos.spectroscopy import trace_detection
from samos.utils import display
import numpy as np
import matplotlib.pyplot as plt

# Read arc lamp
arc_hdu = mosaic.read_sami_mosaic('ARC.fits')
arc = cosmic_rays.remove_cosmic_rays(arc_hdu.data)

# Try different detection parameters
print("Testing different thresholds:")
for threshold in [3e5, 4e5, 5e5]:
    slits = trace_detection.detect_traces_from_arc(
        arc,
        peak_threshold=threshold,
        verbose=False
    )
    print(f"  Threshold {threshold:.1e}: Found {len(slits)} slits")

# Visualize detection
slits = trace_detection.detect_traces_from_arc(arc, peak_threshold=4e5)

# Plot with slit boundaries overlaid
plt.figure(figsize=(16, 8))
vmin, vmax = display.get_percentile_limits(arc, 1, 99)
plt.imshow(arc, origin='lower', cmap='gray', vmin=vmin, vmax=vmax, aspect='auto')

# Overlay slit boundaries
for i, slit in enumerate(slits):
    plt.axhline(slit[0], color='cyan', linewidth=0.5, alpha=0.7)  # Bottom margin
    plt.axhline(slit[1], color='yellow', linewidth=1)              # Trace start
    plt.axhline(slit[2], color='yellow', linewidth=1)              # Trace end
    plt.axhline(slit[3], color='cyan', linewidth=0.5, alpha=0.7)  # Top margin
    
    # Label
    plt.text(100, (slit[1]+slit[2])/2, f'{i}', color='red', fontsize=10)

plt.xlabel('X (dispersion)')
plt.ylabel('Y (spatial)')
plt.title(f'Arc Lamp with {len(slits)} Detected Traces')
plt.colorbar(label='Counts')
plt.show()

### Example: Manual Slit Definition

If automatic detection fails, you can manually define slits:

In [ ]:
# Manual slit definition: [y0, y1_trace, y2_trace, y3]
# Format: [bottom_margin, trace_start, trace_end, top_margin]

manual_slits = [
    [241, 253, 271, 283],   # Slit 0
    [327, 339, 356, 368],   # Slit 1
    [418, 430, 448, 460],   # Slit 2
    # ... add more slits
]

print(f"Defined {len(manual_slits)} slits manually")

# Use these slits for extraction (same as automatic detection output)
slits = manual_slits

### Example: Step-by-Step Detection

See what's happening at each stage:

In [ ]:
# Step 1: Find slit edges
slit_up, slit_down = trace_detection.find_slit_edges(
    arc,
    x_range=(1500, 2000),
    peak_threshold=4e5
)
print(f"Found {len(slit_up)} upper edges and {len(slit_down)} lower edges")

# Step 2: Resolve overlaps
slit_up_clean, slit_down_clean = trace_detection.resolve_overlapping_slits(
    slit_up, slit_down, verbose=True
)

# Step 3: Define regions with margins
slits = trace_detection.define_slit_regions(
    slit_up_clean, slit_down_clean, margin=12
)

print(f"\nFinal: {len(slits)} slits defined")
for i, slit in enumerate(slits[:5]):
    print(f"  Slit {i}: {slit}")

### API Documentation

In [ ]:
from samos.spectroscopy import trace_detection

help(trace_detection.detect_traces_from_arc)
help(trace_detection.find_slit_edges)
help(trace_detection.resolve_overlapping_slits)
help(trace_detection.define_slit_regions)

### Provide Feedback

If you need interactive trace identification features:
- Contact the development team
- Describe your use case
- Share problematic datasets

This will help prioritize implementation of this notebook!